# Due to Some Reason i had to stop working on this project will restart it before buliding an application , will revise pytorch at one shot

# Pytorch at one shot(Regression Based Analysis)


In [24]:
#Import library
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.pytorch


In [25]:
#Load data 
data=pd.read_csv('C:/Users/aditya/Desktop/2023/Deep Learning/ANN_PROJECT_2/Data/AP001.csv')

In [26]:
data.head()

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (mg/m3),...,Temp (degree C),RH (%),WS (m/s),WD (deg),SR (W/mt2),BP (mmHg),VWS (m/s),Xylene (ug/m3),RF (mm),AT (degree C)
0,2016-07-01 10:00:00,2016-07-01 11:00:00,10.67,39.0,17.67,39.2,32.33,7.07,6.60,0.48,...,33.43,71.67,2.30,226.33,123.67,NaN,-0.1,0.1,0.0,23.05
1,2016-07-01 11:00:00,2016-07-01 12:00:00,2.00,39.0,20.50,41.9,35.80,7.40,NaN,0.49,...,33.70,70.00,2.50,223.00,186.00,NaN,-0.1,0.1,0.0,NaN
2,2016-07-01 12:00:00,2016-07-01 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-01 13:00:00,2016-07-01 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-01 14:00:00,2016-07-01 15:00:00,20.50,50.0,15.40,43.6,32.78,6.35,6.38,0.47,...,33.57,63.50,1.88,223.00,240.50,NaN,-0.1,0.1,0.0,NaN


In [27]:
#Quick cleaning
def get_cleaned_data(data : pd.DataFrame)-> pd.DataFrame :
   
    mlflow.set_experiment("Data Cleaning Expirement") 
    with mlflow.start_run():

        mlflow.log_metric("Row_Count_Before_Cleaning",data.shape[0])
        mlflow.log_metric("Column_Count_Before_Cleaning",data.shape[1])
        data.drop(['From Date','To Date','BP (mmHg)'],axis=1,inplace=True)
        data.dropna(inplace=True,axis=0)
        
        mlflow.log_metric("Row_Count_After_Cleaning",data.shape[0])
        mlflow.log_metric("Column_Count_After_Cleaning",data.shape[1])
         

    return data


In [28]:
data_cleaned=get_cleaned_data(data)

INFO: 'Data Cleaning Expirement' does not exist. Creating a new experiment


In [29]:
data_cleaned.head()

,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (mg/m3),Ozone (ug/m3),Benzene (ug/m3),Toluene (ug/m3),Temp (degree C),RH (%),WS (m/s),WD (deg),SR (W/mt2),VWS (m/s),Xylene (ug/m3),RF (mm),AT (degree C)
0,10.67,39.00,17.67,39.20,32.33,7.07,6.60,0.48,14.50,1.00,4.63,33.43,71.67,2.30,226.33,123.67,-0.1,0.10,0.0,23.05
5,15.25,59.50,24.30,45.12,40.12,6.65,6.53,0.51,6.60,0.77,5.33,33.55,63.75,1.40,213.25,152.25,-0.1,0.10,0.0,25.60
6,11.67,60.00,26.73,49.10,43.80,6.93,5.70,0.46,17.43,0.80,6.47,33.57,64.00,1.50,208.67,92.67,-0.1,0.23,0.0,25.70
7,11.75,57.50,19.10,46.33,36.82,8.12,6.23,0.44,19.98,0.70,6.40,33.48,64.00,1.95,222.75,35.00,-0.1,0.10,0.0,25.15
8,18.00,57.75,14.50,44.20,32.33,7.78,6.07,0.45,12.20,0.70,5.30,33.23,63.75,2.15,220.75,14.00,-0.1,0.10,0.0,24.90
